## Accelerate Inference: Neural Network Pruning

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2024-04-15 07:30:30.425407: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-15 07:30:30.475332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 07:30:30.475365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 07:30:30.476548: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-15 07:30:30.484496: I tensorflow/core/platform/cpu_feature_guar

2.15.0


In [2]:
# untar
!ls
# !tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('dataset/train_images.pkl', 'rb'))
train_labels = pickle.load(open('dataset/train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('dataset/val_images.pkl', 'rb'))
val_labels = pickle.load(open('dataset/val_labels.pkl', 'rb'))

CNN_pruning_students_Ava.ipynb	 my_model_weights_2.h5
CNN_pruning_students_qtan.ipynb  my_model_weights_3.h5
README.md			 my_model_weights_glenn.h5
ava_1_weights.h5		 my_model_weights_neuron_pruning.h5
dataset				 original_weights.h5
my_model_weights_1.h5


In [3]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

2024-04-15 07:30:34.102106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20617 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [5]:
# you can use the default hyper-parameters for training,
# val accuracy ~72% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=60,
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/60


2024-04-15 07:30:35.990572: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-15 07:30:37.395556: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900
2024-04-15 07:30:40.283980: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f89f224bff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-15 07:30:40.284018: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
2024-04-15 07:30:40.290474: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713166240.386286   49850 device_compiler.h:186] Compiled cluster using XLA!  This line 

703/703 [==============================] - 11s 6ms/step - loss: 1.5133 - accuracy: 0.3073 - val_loss: 1.3711 - val_accuracy: 0.3917
Epoch 2/60
703/703 [==============================] - 3s 5ms/step - loss: 1.3359 - accuracy: 0.4353 - val_loss: 1.2467 - val_accuracy: 0.4689
Epoch 3/60
703/703 [==============================] - 3s 5ms/step - loss: 1.2669 - accuracy: 0.4658 - val_loss: 1.2181 - val_accuracy: 0.4824
Epoch 4/60
703/703 [==============================] - 3s 5ms/step - loss: 1.2196 - accuracy: 0.4952 - val_loss: 1.1614 - val_accuracy: 0.5244
Epoch 5/60
703/703 [==============================] - 3s 5ms/step - loss: 1.1854 - accuracy: 0.5173 - val_loss: 1.1349 - val_accuracy: 0.5303
Epoch 6/60
703/703 [==============================] - 3s 5ms/step - loss: 1.1566 - accuracy: 0.5307 - val_loss: 1.0863 - val_accuracy: 0.5616
Epoch 7/60
703/703 [==============================] - 3s 5ms/step - loss: 1.1236 - accuracy: 0.5467 - val_loss: 1.0786 - val_accuracy: 0.5620
Epoch 8/60
703/7

In [7]:
results = model.evaluate(val_images, val_labels, batch_size=128)

 1/20 [>.............................] - ETA: 0s - loss: 0.9785 - accuracy: 0.6562

20/20 [==============================] - 0s 3ms/step - loss: 0.7278 - accuracy: 0.7263


In [8]:
model.save_weights("original_weights.h5")

In [9]:
def calculate_model_score(model, accuracy):
    # Initialize counters for zero weights and total parameters
    num_zero_weights = 0
    total_parameters = 0

    # Iterate through all layers of the model
    for layer in model.layers:
        # Get the weights of the layer (this includes both weights and biases)
        weights = layer.get_weights()
        for w in weights:
            # Count zeros and total parameters in each array of weights/biases
            num_zero_weights += np.sum(w == 0)
            total_parameters += np.size(w)

    # Calculate sparsity as the ratio of zero weights to total parameters
    sparsity = num_zero_weights / total_parameters

    # Calculate the score based on the given conditions
    if accuracy > 0.6 and sparsity > 0:
        score = (accuracy + sparsity) / 2
    else:
        score = 0

    print(f"Accuracy: {accuracy}")
    print(f"Sparsity: {sparsity}")
    print(f"Model Score: {score}")

    return score

# Example usage
accuracy = results[1]  # The accuracy you mentioned
score = calculate_model_score(model, accuracy)
print(f"Model score: {score}")

Accuracy: 0.726336658000946
Sparsity: 0.0
Model Score: 0
Model score: 0


In [37]:
model.load_weights('original_weights.h5')

In [38]:
def apply_l1_norm_based_pruning(layer, filter_percentage_threshold):
    weights, biases = layer.get_weights()
    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(weights), axis=(0, 1, 2))
    # Determine the pruning threshold based on the desired percentage
    threshold = np.percentile(l1_norms, filter_percentage_threshold)
    # Create a mask where filters with norm below the threshold are zeroed out
    filter_mask = l1_norms > threshold
    # Apply mask on the filter dimension and broadcast correctly
    new_weights = weights * filter_mask[np.newaxis, np.newaxis, np.newaxis, :]
    layer.set_weights([new_weights, biases])
    
def apply_magnitude_based_pruning(weights, sparsity_level):
    """
    Apply magnitude-based pruning to the weights.
    """
    absolute_weights = np.abs(weights)
    threshold = np.percentile(absolute_weights, sparsity_level)
    weight_mask = absolute_weights > threshold
    return weights * weight_mask

def apply_combined_pruning(model, filter_percentage_threshold, weight_sparsity_level):
    """
    Apply combined L1-norm based filter pruning and magnitude-based pruning to each layer in the model that has weights.
    """
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
            if isinstance(layer, tf.keras.layers.Conv2D):
                # Apply L1-norm based filter pruning only for Conv2D layers
                apply_l1_norm_based_pruning(layer, filter_percentage_threshold)
            # After optional filter pruning, get the updated weights
            weights, biases = layer.get_weights()
            # Apply magnitude-based pruning on the weights and biases
            pruned_weights = apply_magnitude_based_pruning(weights, weight_sparsity_level)
            pruned_biases = apply_magnitude_based_pruning(biases, weight_sparsity_level)
            # Set the pruned weights back to the layer
            layer.set_weights([pruned_weights, pruned_biases])

apply_combined_pruning(model, 0, 70)
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 3ms/step - loss: 1.7317 - accuracy: 0.2760


In [39]:
# check score
accuracy = results[1]
score = calculate_model_score(model, accuracy)
print(f"Model score: {score}")

Accuracy: 0.27603960037231445
Sparsity: 0.6999964582844942
Model Score: 0
Model score: 0


In [ ]:
Draw one graph. In this graph, for each of the following methods, I want to draw a line showing the pareto frontier of acc and sparsity. y axis is acc, x acc is sparsity. (DON'T HAVE TO USE MODEL)

Method 1: magnitude based weight pruning
20:
Accuracy: 0.7124752402305603
Sparsity: 0.20000742073725025
Model Score: 0.45624133048390525
Model score: 0.45624133048390525

30:
Accuracy: 0.6364356279373169
Sparsity: 0.3000035417155058
Model Score: 0.46821958482641135
Model score: 0.46821958482641135

40:
Accuracy: 0.550891101360321
Sparsity: 0.4000047222873411
Model Score: 0
Model score: 0

50:
Accuracy: 0.4534653425216675
Sparsity: 0.5000008432655966
Model Score: 0
Model score: 0

60:
Accuracy: 0.2982178330421448
Sparsity: 0.5999969642438522
Model Score: 0
Model score: 0

70:
Accuracy: 0.27603960037231445
Sparsity: 0.6999964582844942
Model Score: 0
Model score: 0

In [134]:
# model.load_weights('original_weights.h5')

In [137]:
# apply_combined_pruning(model, 5, 0)
# results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 3ms/step - loss: 0.9694 - accuracy: 0.6131


In [138]:
# # check score
# accuracy = results[1]
# score = calculate_model_score(model, accuracy)
# print(f"Model score: {score}")

Accuracy: 0.6130692958831787
Sparsity: 0.4425525312303414
Model Score: 0.52781091355676
Model score: 0.52781091355676


In [40]:
class FreezePrunedWeightsCallback(tf.keras.callbacks.Callback):
    def __init__(self, model):
        super(FreezePrunedWeightsCallback, self).__init__()
        self.model = model
        self.masks = []
        self.original_weights = []

        for layer in self.model.layers:
            if hasattr(layer, 'get_weights') and len(layer.get_weights()) > 0:
                weights, biases = layer.get_weights()
                mask = np.ones_like(weights)
                mask[weights == 0] = 0
                self.masks.append(mask)
                self.original_weights.append(weights * mask)  # Store the pruned weights
            else:
                self.masks.append(None)
                self.original_weights.append(None)

    def on_batch_end(self, batch, logs=None):
        for i, layer in enumerate(self.model.layers):
            if self.masks[i] is not None:
                weights, biases = layer.get_weights()
                pruned_weights = self.original_weights[i]
                weights = pruned_weights  # Reapply the stored pruned weights
                layer.set_weights([weights, biases])

# Usage of the updated callback
freeze_pruned_weights_callback = FreezePrunedWeightsCallback(model)

# Training with callback
model.fit(train_images, train_labels, batch_size=32, epochs=10,
          validation_data=(val_images, val_labels),
          callbacks=[freeze_pruned_weights_callback])

# Evaluating the model
results = model.evaluate(val_images, val_labels, batch_size=128)


Epoch 1/10
703/703 [==============================] - 10s 14ms/step - loss: 1.5687 - accuracy: 0.2975 - val_loss: 1.5185 - val_accuracy: 0.3446
Epoch 2/10
703/703 [==============================] - 10s 14ms/step - loss: 1.4967 - accuracy: 0.3558 - val_loss: 1.4362 - val_accuracy: 0.3865
Epoch 3/10
703/703 [==============================] - 10s 14ms/step - loss: 1.3844 - accuracy: 0.4325 - val_loss: 1.2784 - val_accuracy: 0.5121
Epoch 4/10
703/703 [==============================] - 10s 14ms/step - loss: 1.2473 - accuracy: 0.5127 - val_loss: 1.1643 - val_accuracy: 0.5723
Epoch 5/10
703/703 [==============================] - 10s 14ms/step - loss: 1.1566 - accuracy: 0.5572 - val_loss: 1.1065 - val_accuracy: 0.5925
Epoch 6/10
703/703 [==============================] - 10s 14ms/step - loss: 1.1071 - accuracy: 0.5796 - val_loss: 1.0689 - val_accuracy: 0.6040
Epoch 7/10
703/703 [==============================] - 10s 14ms/step - loss: 1.0771 - accuracy: 0.5948 - val_loss: 1.0436 - val_accuracy:

In [41]:
# check score
accuracy = results[1]
score = calculate_model_score(model, accuracy)
print(f"Model score: {score}")

Accuracy: 0.62851482629776
Sparsity: 0.6991869233117401
Model Score: 0.66385087480475
Model score: 0.66385087480475


In [ ]:
Method 2: magnitude based weight pruning with five more epochs of finetuning
20:
Accuracy: 0.7251484990119934
Sparsity: 0.19976456024542402
Model Score: 0.4624565296287087
Model score: 0.4624565296287087
30:
Accuracy: 0.7302970290184021
Sparsity: 0.29964431057134616
Model Score: 0.5149706697948742
Model score: 0.5149706697948742
40:
Accuracy: 0.7188118696212769
Sparsity: 0.3995257474284616
Model Score: 0.5591688085248692
Model score: 0.5591688085248692
50:
Accuracy: 0.688316822052002
Sparsity: 0.4994105573479634
Model Score: 0.5938636896999827
Model score: 0.5938636896999827
60:
Accuracy: 0.6419801712036133
Sparsity: 0.5992919942050788
Model Score: 0.620636082704346
Model score: 0.620636082704346
70:
Accuracy: 0.62851482629776
Sparsity: 0.6991869233117401
Model Score: 0.66385087480475
Model score: 0.66385087480475

    

In [142]:
model.save_weights("ava_1_weights.h5")

In [173]:
model.load_weights("ava_1_weights.h5")

In [174]:
apply_combined_pruning(model, 0, 49)
results = model.evaluate(val_images, val_labels, batch_size=128)


 1/20 [>.............................] - ETA: 0s - loss: 1.2638 - accuracy: 0.4844

20/20 [==============================] - 0s 3ms/step - loss: 0.9540 - accuracy: 0.6206


In [175]:
# check score
accuracy = results[1]
score = calculate_model_score(model, accuracy)
print(f"Model score: {score}")

Accuracy: 0.6205940842628479
Sparsity: 0.4899980267585039
Model Score: 0.5552960555106758
Model score: 0.5552960555106758


In [176]:
apply_combined_pruning(model, 7, 0)
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 3ms/step - loss: 1.0245 - accuracy: 0.6111


In [177]:
# check score
accuracy = results[1]
score = calculate_model_score(model, accuracy)
print(f"Model score: {score}")

Accuracy: 0.6110891103744507
Sparsity: 0.4909913936313209
Model Score: 0.5510402520028858
Model score: 0.5510402520028858


In [178]:
# Training with callback
model.fit(train_images, train_labels, batch_size=32, epochs=5,
          validation_data=(val_images, val_labels),
          callbacks=[freeze_pruned_weights_callback])

# Evaluating the model
results = model.evaluate(val_images, val_labels, batch_size=128)

Epoch 1/5
703/703 [==============================] - 9s 13ms/step - loss: 0.6888 - accuracy: 0.7399 - val_loss: 0.7703 - val_accuracy: 0.7061
Epoch 2/5
703/703 [==============================] - 9s 13ms/step - loss: 0.6787 - accuracy: 0.7435 - val_loss: 0.7656 - val_accuracy: 0.7077
Epoch 3/5
703/703 [==============================] - 9s 13ms/step - loss: 0.6821 - accuracy: 0.7437 - val_loss: 0.7657 - val_accuracy: 0.7081
Epoch 4/5
703/703 [==============================] - 9s 13ms/step - loss: 0.6776 - accuracy: 0.7471 - val_loss: 0.7604 - val_accuracy: 0.7093
Epoch 5/5
20/20 [==============================] - 0s 2ms/step - loss: 0.7608 - accuracy: 0.7137


In [179]:
# check score
accuracy = results[1]
score = calculate_model_score(model, accuracy)
print(f"Model score: {score}")

Accuracy: 0.7136633396148682
Sparsity: 0.4420516314659498
Model Score: 0.5778574855404089
Model score: 0.5778574855404089


In [182]:
# model.load_weights("my_model_weights_glenn.h5")
# results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 3ms/step - loss: 0.7084 - accuracy: 0.7461


In [183]:
# # check score
# accuracy = results[1]
# score = calculate_model_score(model, accuracy)
# print(f"Model score: {score}")

Accuracy: 0.7461386322975159
Sparsity: 0.6957008633353178
Model Score: 0.7209197478164169
Model score: 0.7209197478164169


In [1]:
import numpy as np

# Example weights tensor (random data for illustration)
weights = np.random.randn(3, 3, 1, 64)  # A common shape for conv layer weights

# Calculate absolute values
absolute_weights = np.abs(weights)

# Determine the 10th percentile
sparsity_level = 10  # We want to prune the lowest 10% of weights by magnitude
threshold = np.percentile(absolute_weights, sparsity_level)

# Show the threshold
print(f"The threshold for pruning 10% of weights is: {threshold}")


The threshold for pruning 10% of weights is: 0.10879103297130306
